In [16]:
# Import the required libraries
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import numpy as np
import pandas as pd
import math
import json
import time
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
import joblib
import scipy.sparse
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds
import warnings; warnings.simplefilter('ignore')
%matplotlib inline

Loading dataset and add headers

In [3]:
# Import the dataset and give the column names
columns = ['userId', 'productId', 'ratings','timestamp']
electronics_df = pd.read_csv('Dataset.csv', names=columns)

In [ ]:
electronics_df.head()

In [ ]:
# Removing the timestamp column
electronics_df.drop('timestamp',
                    axis = 1,
                    inplace = True
                    )

In [ ]:
electronics_df.info()

In [ ]:
# Check the number of rows and columns
rows, columns = electronics_df.shape
print('Number of rows: ', rows)
print('Number of columns: ', columns)

In [ ]:
# Check the datatypes
electronics_df.dtypes

#~ Output:
    #~ userId        object
    #~ productId     object
    #~ ratings       float64
    #~ dtype:        object

In [9]:
# Taking subset of the dataset
electronics_df1 = electronics_df.iloc[:50000,0:]      # Done due to size of Dataset i.e. 374 million columns

In [ ]:
electronics_df1.info()

In [ ]:
# Summary statistics of rating variable
electronics_df1['ratings'].describe().transpose()

In [ ]:
# Find the minimum and maximum ratings
print('Minimum rating is: %d' %(electronics_df1.ratings.min()))
print('Maximum rating is: %d' %(electronics_df1.ratings.max()))

#~ Rating scale is from 1 to 5

Handelling missing values

In [ ]:
# Check for missing values
print('Number of missing values across columns: \n',electronics_df.isnull().sum())

 Ratings

In [ ]:
# Check the distribution of the rating
with sns.axes_style('white'):
    g = sns.factorplot("ratings", 
                       data = electronics_df1, 
                       aspect = 2.0, 
                       kind = 'count'
                       )
    g.set_ylabels("Total number of ratings")

#~ Done to see which rating is given by most number of users


Users and Products

In [ ]:
# Number of unique user id  in the data
print('Number of unique users in Raw data = ', electronics_df1['userId'].nunique())
# Number of unique product id  in the data
print('Number of unique product in Raw data = ', electronics_df1['productId'].nunique())

Taking the subset of dataset to reduce density and make it sparse

In [ ]:
# Check the top 10 users based on ratings
most_rated = electronics_df1.groupby('userId').size().sort_values(ascending = False)[:10]
print('Top 10 users based on ratings: \n',most_rated)

In [ ]:
counts = electronics_df1.userId.value_counts()
electronics_df1_final = electronics_df1[electronics_df1.userId.isin(counts[counts >= 15].index)]
print('Number of users who have rated 25 or more items = ', len(electronics_df1_final))
print('Number of unique users in the final data = ', electronics_df1_final['userId'].nunique())
print('Number of unique products in the final data = ', electronics_df1_final['userId'].nunique())

#~ electronics_df1_final has the users who have rated 25 or more different items

Rating analysis in the final dataset

In [ ]:
# Constructing the pivot table
final_ratings_matrix = electronics_df1_final.pivot(index = 'userId', columns = 'productId', values = 'ratings').fillna(0)
final_ratings_matrix.head()

#~ Will be a sparse matrix if most of the cells are filled with 0 as values

In [ ]:
print('Shape of final_ratings_matrix: ', final_ratings_matrix.shape)

In [ ]:
# Calucating the density of the rating marix
given_num_of_ratings = np.count_nonzero(final_ratings_matrix)
print('given_num_of_ratings = ', given_num_of_ratings)
possible_num_of_ratings = final_ratings_matrix.shape[0] * final_ratings_matrix.shape[1]
print('possible_num_of_ratings = ', possible_num_of_ratings)
density = (given_num_of_ratings/possible_num_of_ratings)
density *= 100
print ('density: {:4.2f}%'.format(density))

#~ If density comes out to be less than 20%, then it will be a sparse matrix

Splitting data for TRAINING & TESTING

In [ ]:
# Split the data randomnly into train and test datasets into 70:30 ratio
train_data, test_data = train_test_split(electronics_df1_final, 
                                         test_size = 0.3, 
                                         random_state = 0
                                         )
train_data.head()

In [ ]:
print('Shape of training data: ', train_data.shape)
print('Shape of testing data: ', test_data.shape)

Building Popularity Recommender model

In [ ]:
# Count of user_id for each unique product as recommendation score 
train_data_grouped = train_data.groupby('productId').agg({'userId': 'count'}).reset_index()
train_data_grouped.rename(columns = {'userId': 'score'}, inplace = True)
train_data_grouped.head(40)

In [ ]:
#~ Generating the recommendation order

# Sort the products on recommendation score 
train_data_sort = train_data_grouped.sort_values(['score', 'productId'], ascending = [0,1]) 
      
# Generate a recommendation rank based upon score 
train_data_sort['rank'] = train_data_sort['score'].rank(ascending=0, method='first') 
          
# Get the top 5 recommendations 
popularity_recommendations = train_data_sort.head(5) 
popularity_recommendations 

Popularity based recommender model

In [ ]:
def recommend(user_id):     
    user_recommendations = popularity_recommendations 
          
    # Add user_id column for which the recommendations are being generated 
    user_recommendations['userId'] = user_id 
      
    # Bring user_id column to the front 
    cols = user_recommendations.columns.tolist() 
    cols = cols[-1:] + cols[:-1] 
    user_recommendations = user_recommendations[cols] 
          
    return user_recommendations 

In [ ]:
find_recom = [10,100,150]   # This list is user choice.
for i in find_recom:
    print("The list of recommendations for the userId: %d\n" %(i))
    print(recommend(i))    
    print("\n")
    
#~ Since, it is a Popularity recommender model, so, all the three users are given the same recommendations.
#~ Here, we predict the products based on the popularity. It is not personalized to particular user.
#~ It is a non-personalized recommender system.

Collaborative Filtering recommender model i.e. User Based Collaborative Filtering model

In [ ]:
electronics_df_CF = pd.concat([train_data, test_data]).reset_index()
electronics_df_CF.head()

In [ ]:
# Matrix with row per 'user' and column per 'item' 
pivot_df = electronics_df_CF.pivot(index = 'userId', 
                                   columns = 'productId', 
                                   values = 'ratings').fillna(0)
pivot_df.head()

In [ ]:
print('Shape of the pivot table: ', pivot_df.shape)

In [ ]:
# Define user index from 0 to 10
pivot_df['user_index'] = np.arange(0, pivot_df.shape[0], 1)
pivot_df.head()

In [ ]:
pivot_df.set_index(['user_index'], 
                   inplace = True
                   )

# Actual ratings given by users
pivot_df.head()

#~ As this matrix is a sparse matrix, we will be using Singular Value Decomposition

Singular Value Decomposition

In [ ]:
# Singular Value Decomposition
U, sigma, Vt = svds(pivot_df, k = 10)

In [ ]:
print('Left singular matrix: \n',U)

In [ ]:
print('Sigma: \n',sigma)
#~ As sigma is not a diagonal matrix, we have to convert it into a diagonal one

In [ ]:
# Construct diagonal array in SVD
sigma = np.diag(sigma)              # Matrix transformation function
print('Diagonal matrix: \n',sigma)

In [ ]:
print('Right singular matrix: \n',Vt)

In [ ]:
# Predicted ratings
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) 
# Convert predicted ratings to dataframe
preds_df = pd.DataFrame(all_user_predicted_ratings, columns = pivot_df.columns)
preds_df.head()

In [ ]:
# Recommend the items with the highest predicted ratings

def recommend_items(userID, pivot_df, preds_df, num_recommendations):
    # index starts at 0  
    user_idx = userID-1 
    # Get and sort the user's ratings
    sorted_user_ratings = pivot_df.iloc[user_idx].sort_values(ascending = False)
    # sorted_user_ratings
    sorted_user_predictions = preds_df.iloc[user_idx].sort_values(ascending = False)
    # sorted_user_predictions
    temp = pd.concat([sorted_user_ratings, sorted_user_predictions], axis = 1)
    temp.index.name = 'Recommended Items'
    temp.columns = ['user_ratings', 'user_predictions']
    temp = temp.loc[temp.user_ratings == 0]   
    temp = temp.sort_values('user_predictions', ascending = False)
    
    print('\nBelow are the recommended items for user(user_id = {}):\n'.format(userID))
    print(temp.head(num_recommendations))

In [ ]:
# Recommended items for user with user_id = 4
userID = 4
num_recommendations = 5
recommend_items(userID, pivot_df, preds_df, num_recommendations)

In [ ]:
# Below are the recommended items for user with user_id = 6
userID = 6
num_recommendations = 5
recommend_items(userID, pivot_df, preds_df, num_recommendations)

In [ ]:
# Below are the recommended items for user with user_id = 8
userID = 8
num_recommendations = 5
recommend_items(userID, pivot_df, preds_df, num_recommendations)

#~ Since, it is a Collaborative recommender model, so, all the three users are given different recommendations based on users past behaviour.

Evaluation of Collabrative recommendation model

In [ ]:
# Actual ratings given by the users
final_ratings_matrix.head()

In [ ]:
# Average ACTUAL rating for each item
final_ratings_matrix.mean().head()

In [ ]:
# Predicted ratings 
preds_df.head()

In [ ]:
# Average PREDICTED rating for each item
preds_df.mean().head()

In [ ]:
rmse_df = pd.concat([final_ratings_matrix.mean(), preds_df.mean()], axis = 1)
rmse_df.columns = ['Avg_actual_ratings', 'Avg_predicted_ratings']
print(rmse_df.shape)
rmse_df['item_index'] = np.arange(0, rmse_df.shape[0], 1)
rmse_df.head()

In [ ]:
RMSE = round((((rmse_df.Avg_actual_ratings - rmse_df.Avg_predicted_ratings) ** 2).mean() ** 0.5), 5)
print('\nRMSE SVD Model = {} \n'.format(RMSE))

RMSE SVD Model = 0.05854

Getting top - K ( K = 5) recommendations

In [ ]:
# Enter 'userID' and 'num_recommendations' for the user
userID = 9
num_recommendations = 5
recommend_items(userID, pivot_df, preds_df, num_recommendations)
# Below are the recommended items for user(user_id = 9):

Conclusions and Inferences:

1. The Popularity-based recommender system is a non-personalised recommender system and these are based on frequecy counts, which may be not suitable to the user. We can see the differance above for the user id 4, 6 & 8, The Popularity based model has recommended the same set of 5 products to both but Collaborative Filtering based model has recommended entire different list based on the user past purchase history.

2. Model-based Collaborative Filtering is a personalised recommender system, the recommendations are based on the past behavior of the user and it is not dependent on any additional information.